In [1]:
# NOTE: you CAN change this cell
# If you want to use your own database, download it here
# Download DB
!rm -rf list_province.txt list_district.txt list_ward.txt
# this link is public test
!gdown --fuzzy https://drive.google.com/file/d/15vyI38GkCs40Vv9Yho5C5jWO9TsXimJ5/view?usp=drive_link -O list_ward.txt
!gdown --fuzzy https://drive.google.com/file/d/1kPo0d2mqrmlXDiWirLPd6ZFHWDXNRUGV/view?usp=drive_link -O list_district.txt
!gdown --fuzzy https://drive.google.com/file/d/1YD1S15-BMmooS7yQyTj8vMA4yiWxNVY2/view?usp=drive_link -O list_province.txt

Downloading...
From: https://drive.google.com/uc?id=15vyI38GkCs40Vv9Yho5C5jWO9TsXimJ5
To: /content/list_ward.txt
100% 4.18k/4.18k [00:00<00:00, 11.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kPo0d2mqrmlXDiWirLPd6ZFHWDXNRUGV
To: /content/list_district.txt
100% 3.04k/3.04k [00:00<00:00, 9.00MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YD1S15-BMmooS7yQyTj8vMA4yiWxNVY2
To: /content/list_province.txt
100% 728/728 [00:00<00:00, 2.68MB/s]


In [2]:
# NOTE: you CAN change this cell
# Add more to your needs
# you must place ALL pip install here
!pip install editdistance
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.8 MB/s eta 0:00:00


In [3]:
# NOTE: you CAN change this cell
# import your library here
import time
import re
import queue

# V1 (Raw Trie + Levenshtein)

In [ ]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char == ' ': continue
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def search(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                return False
            node = node.children[char]
        return node.is_end_of_word

    def _search_with_distance(self, node, word, index, current_distance, max_distance, current_word, results):
        if current_distance > max_distance:
            return

        if node.is_end_of_word:
            if (current_word != "") and (not current_word.startswith("_")):
              results.append((current_word, current_distance))
            else:
              results.append(("", current_distance))

        if index < len(word):
            for char, child_node in node.children.items():
                # Check for substitution
                if char == word[index]:
                    self._search_with_distance(child_node, word, index + 1, current_distance, max_distance, current_word + char, results)
                else:
                    # Check for substitution and insertion
                    self._search_with_distance(child_node, word, index + 1, current_distance + 1, max_distance, current_word + char, results)

                    # Check for deletion
                    self._search_with_distance(child_node, word, index, current_distance + 1, max_distance, current_word + char, results)

        # If we are still within the length of the word, add the character to the current word (insert case)
        if index < len(word):
            self._search_with_distance(node, word, index + 1, current_distance + 1, max_distance, current_word, results)

    def _search_with_distance_dp(self, node, word, index, current_distance, max_distance, current_word, results, memo):
        # If we have already computed this state, return from memo
        if (index, current_distance, current_word) in memo:
            return memo[(index, current_distance, current_word)]

        # If the current distance exceeds the allowed max distance, stop the recursion
        if current_distance > max_distance:
            return

        # If this is the end of a word in the Trie, add the current word and distance to the results
        if node.is_end_of_word:
            if (current_word != "") and (not current_word.startswith("_")):
                results.append((current_word, current_distance))
            else:
                results.append(("", current_distance))

        # If we still have characters left in the word to process
        if index < len(word):
            for char, child_node in node.children.items():
                # Substitution (or match)
                if char == word[index]:
                    self._search_with_distance_dp(child_node, word, index + 1, current_distance, max_distance, current_word + char, results, memo)
                else:
                    # Substitution with cost (mismatch)
                    self._search_with_distance_dp(child_node, word, index + 1, current_distance + 1, max_distance, current_word + char, results, memo)

                    # Deletion (skipping a character in the word)
                    self._search_with_distance_dp(child_node, word, index, current_distance + 1, max_distance, current_word + char, results, memo)

        # Insertion (adding a character from the Trie without advancing in the word)
        if index < len(word):
            self._search_with_distance_dp(node, word, index + 1, current_distance + 1, max_distance, current_word, results, memo)

        # Memoize the result for this state
        memo[(index, current_distance, current_word)] = results

    def levenshtein_with_trie(self, word, max_distance, dp=False):
        results = []
        if dp:
            self._search_with_distance_dp(self.root, word, 0, 0, max_distance, "", results, {})
        else:
            self._search_with_distance(self.root, word, 0, 0, max_distance, "", results)
        results.sort(key=lambda x: x[1])
        return results

    def _print_trie(self, node, word):
        if node.is_end_of_word:
            print(word)

        for char, child_node in node.children.items():
            self._print_trie(child_node, word + char)

    def print_trie(self):
        print("Words in the Trie:")
        self._print_trie(self.root, "")

In [ ]:
# NOTE: you MUST change this cell
# New methods / functions must be written under class Solution.
class Solution:
    def __init__(self):
        # list provice, district, ward for private test, do not change for any reason

        self.province_path = 'list_province.txt'
        self.district_path = 'list_district.txt'
        self.ward_path = 'list_ward.txt'
        self.root_district = Trie()
        self.root_province = Trie()
        self.root_ward = Trie()

        # Adjustables
        self.max_distance = 3
        self.stopwords = ['thành phố', 'thị trấn', 'phường', 'huyện', 'tỉnh', 'quận', 'tp', 't.p', 'Tp.', 'tt.', 'tx.', 't.x', 'xã', 'h.', 'x.', 't.', 'q.', 'tx', 'x2']

        self.province_data = None
        self.district_data = None
        self.ward_data = None

        self.read_file()
        self.build_tree()

    def read_file(self):
        self.province_data = open(self.province_path, encoding='utf8')
        self.district_data = open(self.district_path, encoding='utf8')
        self.ward_data = open(self.ward_path, encoding='utf8')

    def build_tree(self):
        for i in self.district_data:
            self.root_district.insert(i.replace('\n', ''))
        for i in self.province_data:
            self.root_province.insert(i.replace('\n', ''))
        for i in self.ward_data:
            self.root_ward.insert(i.replace('\n', ''))
        # self.root_ward.print_trie()
        # self.root_province.print_trie()
        # self.root_district.print_trie()

    def remove_stopwords_from_address(self, address: str):
        chunks = address.split(",")
        clear_chunks = []

        for chunk in chunks:
            words:str = chunk.split(" ")
            stopword_clear_words = []
            for word in words:
                for stopword in self.stopwords:
                    if stopword is word:
                        break
                    elif word.lower().startswith(stopword):
                        word = word[len(stopword):]
                    elif word.lower().endswith(stopword):
                        word = word[:len(word) - len(stopword)]
                striped_word = re.sub(r'\W+', '', word.strip())
                if len(striped_word) > 1:
                    stopword_clear_words.append(striped_word)
            clear_chunks.append("".join(stopword_clear_words))
            # TODO: Remove First Letter of Stopword (X, T, H) that follows with a Constant
        # print("After sanitizing: ", clear_chunks)
        return clear_chunks

    def find_nearest_string(self, trie: Trie, text: str, dp=False):
        result = trie.levenshtein_with_trie(text, self.max_distance, dp)
        filtered_result = list(filter(lambda item: len(item[0]) > 1, result))
        if filtered_result:
            nearest_string = filtered_result[0]
        else:
            # Handle the case where no suitable string is found
            nearest_string = ("", float('inf'))  # or any other appropriate default
        return nearest_string

    def add_space_to_pascal_case(self, text):
        # Use regex to find uppercase letters and add a space before each (except the first one)
        spaced_text = re.sub(r'(?<!^)([A-Z])', r' \1', text)
        return spaced_text

    def process(self, s: str):
        # write your process string here
        sanitized_address = self.remove_stopwords_from_address(s)
        # TODO: code to split string
        ward = self.find_nearest_string(self.root_province, sanitized_address[0], dp=True)[0] if len(sanitized_address) > 0 else ""
        district = self.find_nearest_string(self.root_district, sanitized_address[1], dp=True)[0] if len(sanitized_address) > 1 else ""
        province = self.find_nearest_string(self.root_province, sanitized_address[2], dp=True)[0] if len(sanitized_address) > 2 else ""
        result = {
            "province": self.add_space_to_pascal_case(province),
            "district": self.add_space_to_pascal_case(district),
            "ward": self.add_space_to_pascal_case(ward),
        }
        # print("Search result: ", result)
        return result

# V2 (BK-Tree)

In [4]:
MAXN = 10000
TOL = 5
LEN = 10

class Node:
    def __init__(self, x=None):
        self.word = x
        self.next = [0] * 2000

class BKTree:
    def __init__(self, maxn=MAXN, tol=TOL, len=LEN):
        self.root = Node()
        self.ptr = 0
        self.tree = [Node() for _ in range(MAXN)]

    def add(self, curr):
      def _add(root, curr):
        if not root.word:
            root.word = curr.word
            root.next = curr.next
            return

        dist = edit_distance(curr.word, root.word)
        if not self.tree[root.next[dist]] or not self.tree[root.next[dist]].word:
            self.ptr += 1
            self.tree[self.ptr] = curr
            root.next[dist] = self.ptr
        else:
            _add(self.tree[root.next[dist]], curr)

      return _add(self.root, curr)

    def search(self, s):
      def _search(root, s):
        ret = []
        if not root or not root.word:
            return ret

        dist = edit_distance(root.word, s)
        if dist <= TOL:
            ret.append(root.word)

        start = dist - TOL if dist - TOL > 0 else 1
        while start <= dist + TOL:
            tmp = _search(self.tree[root.next[start]], s)
            ret += tmp
            start += 1
        return ret # return pair (word, distance)

      return _search(self.root, s)

def edit_distance(a, b):
    m, n = len(a), len(b)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
          if a[i - 1] != b[j - 1]:
                dp[i][j] = min(
                    dp[i - 1][j] + 1,  # deletion
                    dp[i][j - 1] + 1,  # insertion
                    dp[i - 1][j - 1] + 2  # replacement
                )
          else:
                dp[i][j] = dp[i - 1][j - 1]
    return dp[m][n]

#dictionary words
if __name__ == "__main__":
    tree = BKTree()
    dictionary = ["hell", "help", "shell", "smell", "fell", "felt", "oops", "pop", "oouch", "halt"]
    sz = len(dictionary)
    #adding dict[] words on to tree
    for i in range(sz):
        tmp = Node(dictionary[i])
        tree.add(tmp)

    w1 = "ops"
    match1 = tree.search(w1)
    print("similar words in dictionary for", w1, ": ")
    for word in match1:
        print(word, end=" ")
    print()

    w2 = "helt"
    match2 = tree.search(w2)
    print("similar words in dictionary for", w2, ": ")
    for word in match2:
        print(word, end=" ")
    print()


similar words in dictionary for ops : 
help pop oops 
similar words in dictionary for helt : 
hell shell help fell smell felt halt 


In [5]:
import itertools

def generate_initial_combinations(text):
    """Generates all possible initial combinations of a string.

    Args:
        text: The input string.

    Returns:
        A list of all possible initial combinations.
    """
    words = text.split()

    # Generate all combinations of full words and initials for each word
    word_combinations = []
    for word in words:
        word_combinations.append([word, word[0] + "."])  # Add full word and initial with period

    # Generate all possible combinations using itertools.product
    all_combinations = list(itertools.product(*word_combinations))

    # Join the combinations into strings and return
    initial_combinations = ["".join(comb) for comb in all_combinations]
    return initial_combinations

In [107]:
# NOTE: you MUST change this cell
# New methods / functions must be written under class Solution.
import re
import string
import unidecode

class Solution:
    def __init__(self):
        # list provice, district, ward for private test, do not change for any reason

        self.province_path = 'list_province.txt'
        self.district_path = 'list_district.txt'
        self.ward_path = 'list_ward.txt'
        self.root_district = BKTree()
        self.root_province = BKTree()
        self.root_ward = BKTree()

        # Adjustables
        self.max_distance = 3
        self.stopwords = ['tha6nhpho', 'thanhpho', 'thanh pho', 'thi xa', 'thixa' ,'thi tran', 'phuong', 'huyen', "hyen", "huyn", 'tinh', 'quan ', 'tp', 't.p', 'tp.', 'tt.', 'tx.', 't.x.', 't.x', 'tnh', 'tg.', 'tin', 'tt', 'tt', 'xa', 'h.', 'x.', 't.', 'q.', 'tx', 'x2', 'h ']
        province_sw = ['tha6nhpho', "thanhmpho", "thanhpho", "thanhfho", "tinh", 't.pho', "tp", "t.y", "ty.", "t.p", "Tp.", "tnh", "t.", "tin", "tnh", 'tg.']
        province_sw.sort(key=lambda x: len(x), reverse=True)
        district_sw = ["thanh pho", 'tha6nhpho', "thanhfho", "thanhpho", "tp", 't.pho', "t.p", "Tp.", "tnh", "huyen", "hyen", "huyn", "quan", "q.", "h.", 'thi xa', 'thixa', 'hixa', 'ixa']
        district_sw.sort(key=lambda x: len(x), reverse=True)
        ward_sw = ["phuong", "phg", "ph", "p.", "xa", "x.", "thitran", "tt", "tt.", "thitr", "thtran", "thtr", "f."]
        ward_sw.sort(key=lambda x: len(x), reverse=True)
        self.stopwords_dict = {
            "province": province_sw,
            "district": district_sw,
            "ward": ward_sw
        }

        self.province_sanitized_map = dict()
        self.district_sanitized_map = dict()
        self.ward_sanitized_map = dict()
        self.whitelisted_province = []
        self.whitelisted_district = []
        self.whitelisted_ward = []

        self.read_file()
        self.build_tree()

    def remove_special_characters(self, text, remove_space=True, lowercase=True):
        text = unidecode.unidecode(text)  # Loại bỏ dấu
        # pattern = f"[{re.escape(string.punctuation)}]"
        # text = re.sub(pattern, "", text)  # Loại bỏ ký tự đặc biệt
        if remove_space: text = re.sub(r'\s+', '', text)  # Loại bỏ khoảng trắng
        if lowercase: text = text.lower()               # In thường
        return text

    def read_file(self):
        province_data = open(self.province_path, encoding='utf8')
        district_data = open(self.district_path, encoding='utf8')
        ward_data = open(self.ward_path, encoding='utf8')
        district_prefixes = ["quan", "q", "qn", "q.", "qn."]
        ward_prefixes = ["phuong", "p", "f", "p.", "f."]

        for line in province_data:
            data = line.replace('\n', '')
            self.province_sanitized_map[self.remove_special_characters(data)] = data
            initials = generate_initial_combinations(self.remove_special_characters(data, remove_space=False))
            for initial in initials:
              self.province_sanitized_map.setdefault(initial, data)

        for line in district_data:
            data = line.replace('\n', '')
            s_data = self.remove_special_characters(data)
            self.district_sanitized_map[s_data] = data
            if data.isdigit():
              for prefix in district_prefixes:
                self.district_sanitized_map.update({f"{prefix}{s_data}": data})

        for line in ward_data:
            data = line.replace('\n', '')
            s_data = self.remove_special_characters(data)
            self.ward_sanitized_map[self.remove_special_characters(data)] = data
            if data.isdigit():
              for prefix in ward_prefixes:
                self.ward_sanitized_map.update({f"{prefix}{s_data}": data})
        province_custom_map = {
            "t.t.h": "Thừa Thiên Huế",
            "thua.t.hue": "Thừa Thiên Huế",
            "thua.thien.hue": "Thừa Thiên Huế",
            "hcm": "Hồ Chí Minh",
            "ho.chi.minh": "Hồ Chí Minh",
            "h.c.m": "Hồ Chí Minh",
            "h.c.m.": "Hồ Chí Minh"
        }
        self.province_sanitized_map.update(province_custom_map)


    def build_tree(self):
        for district in self.district_sanitized_map.keys():
            district_node = Node(self.remove_special_characters(district))
            self.root_district.add(district_node)
        for province in self.province_sanitized_map.keys():
            province_node = Node(self.remove_special_characters(province))
            self.root_province.add(province_node)
        for ward in self.ward_sanitized_map.keys():
            ward_node = Node(self.remove_special_characters(ward))
            self.root_ward.add(ward_node)


    def remove_stopwords_from_address(self, address: str):
        chunks = address.lower().split(",")

        clear_chunks = []

        # CONSTANTS
        HEAD_SW_SIZE = 7
        DIST_ALLOWED = 2
        WINDOW_ALLOWED = 9
        OFFSET = 2

        self.whitelisted_province = filter(lambda x: len(x) >= WINDOW_ALLOWED - DIST_ALLOWED, self.province_sanitized_map.keys())
        self.whitelisted_district = filter(lambda x: len(x) >= WINDOW_ALLOWED - DIST_ALLOWED, self.district_sanitized_map.keys())
        self.whitelisted_ward = filter(lambda x: len(x) >= WINDOW_ALLOWED - DIST_ALLOWED, self.ward_sanitized_map.keys())

        def should_elim_prefix_stopword(chunk, stopword):
            head = chunk[:HEAD_SW_SIZE]
            return (len(chunk) >= 1.75*len(stopword) or chunk[len(stopword):].isdigit()) and (chunk.startswith(stopword) or edit_distance(head, stopword) <= DIST_ALLOWED)

        # A. COMMA >= 2
        if len(chunks) >= 3:
            for i in range(len(chunks) - 1, -1, -1):
                # Basic sanitization
                chunk = self.remove_special_characters(chunks[i])
                # Remove stopwords
                for stopword in self.stopwords:
                    if should_elim_prefix_stopword(chunk, stopword):
                        chunk = chunk[len(stopword):]
                        break
                # We are in the Province chunk
                if i == 2:
                    jump_to_next_chunk = False
                    # It is longer than expected -> may contain District
                    if len(chunk) > WINDOW_ALLOWED:
                        is_whitelist = False
                        # Sanit. A.1: Contains Whitelist word
                        for wl_province in self.whitelisted_province:
                            if edit_distance(chunk, wl_province) <= DIST_ALLOWED:
                                is_whitelist = True
                                province = self.remove_special_characters(wl_province)
                                clear_chunks.append(province)
                                break
                        if is_whitelist: continue
                        # Sanit. A.2: Contains Stop word
                        for sw_district in self.stopwords_dict["province"]:
                            if sw_district in chunk:
                                jump_to_next_chunk = True
                                sp = chunk.split(sw_district)
                                province = sp[1]
                                district = sp[0]
                                if len(province) < 4:
                                  clear_chunks.append("")
                                  break
                                clear_chunks.append(province)
                                if len(district) < 3 and not district.isdigit(): break
                                clear_chunks.append(district) if len(district) <= WINDOW_ALLOWED else clear_chunks.append(district[-WINDOW_ALLOWED:])
                                break
                        if jump_to_next_chunk: continue
                        # Sanit. A.3: Not Whitelisted or Stop Word
                        province = self.remove_special_characters(chunk[-len(chunk)//2 - OFFSET:])
                        district = self.remove_special_characters(chunk[:-len(chunk)//2 + OFFSET])
                        if province != district:
                            jump_to_next_chunk = True
                            if len(province) < 3: continue
                            clear_chunks.append(province)
                            if len(district) < 3 and not district.isdigit(): continue
                            clear_chunks.append(district) if len(district) <= WINDOW_ALLOWED else clear_chunks.append(district[-WINDOW_ALLOWED:])
                            if len(clear_chunks) == 3: return clear_chunks
                    # Normal length
                    else:
                        clear_chunks.append(chunk)
                # We are in the District chunk
                if i == 1:
                    jump_to_next_chunk = False
                    # It is longer than expected -> may contain Ward
                    if len(chunk) > WINDOW_ALLOWED:
                        is_whitelist = False
                        # Sanit. A.1: Contains Whitelist word
                        for wl_district in self.whitelisted_district:
                            if edit_distance(chunk, wl_district) <= DIST_ALLOWED:
                                is_whitelist = True
                                district = self.remove_special_characters(wl_district)
                                clear_chunks.append(district)
                                break
                        if is_whitelist: continue
                        # Sanit. A.2: Contains Stop word
                        for sw_district in self.stopwords_dict["district"]:
                            if sw_district in chunk:
                                sp = chunk.split(sw_district)
                                district = self.remove_special_characters(sp[1])
                                ward = self.remove_special_characters(sp[0])
                                if len(district) < 3 and not district.isdigit(): break
                                jump_to_next_chunk = True
                                clear_chunks.append(district)
                                if len(ward) < 3 and not ward.isdigit(): break
                                clear_chunks.append(ward) if len(ward) <= WINDOW_ALLOWED else clear_chunks.append(ward[-WINDOW_ALLOWED:])
                                if len(clear_chunks) == 3: return clear_chunks
                                break
                        if jump_to_next_chunk: continue
                        # Sanit. A.3: Not Whitelisted or Stop Word
                        district = self.remove_special_characters(chunk[-len(chunk)//2 - OFFSET:])
                        ward = self.remove_special_characters(chunk[:-len(chunk)//2 + OFFSET])
                        if district != ward:
                            jump_to_next_chunk = True
                            if len(district) < 3 and not district.isdigit(): continue
                            clear_chunks.append(district)
                            if len(ward) < 3 and not ward.isdigit(): continue
                            clear_chunks.append(ward) if len(ward) <= WINDOW_ALLOWED else clear_chunks.append(ward[-WINDOW_ALLOWED:])
                            if len(clear_chunks) == 3: return clear_chunks
                    # Normal length, but be careful
                    else:
                        for sw_district in self.stopwords_dict["district"]:
                            if sw_district in chunk:
                                sp = chunk.split(sw_district)
                                district = self.remove_special_characters(sp[1])
                                ward = self.remove_special_characters(sp[0])
                                if len(district) < 3 and not district.isdigit(): break
                                jump_to_next_chunk = True
                                clear_chunks.append(district)
                                if len(ward) < 3 and not ward.isdigit(): break
                                clear_chunks.append(ward) if len(ward) <= WINDOW_ALLOWED else clear_chunks.append(ward[-WINDOW_ALLOWED:])
                                if len(clear_chunks) == 3: return clear_chunks
                                break
                        district = self.remove_special_characters(chunk)
                    if not jump_to_next_chunk: clear_chunks.append(district)
                    continue
                # We are in the Ward chunk
                if len(clear_chunks) == 3: return clear_chunks
                if i == 0:
                    # It is longer than expected, may contain house numbers
                    if len(chunk) > WINDOW_ALLOWED:
                        is_whitelist = False
                        word, min_dist = "", float('inf')
                        for wl_ward in self.whitelisted_ward:
                            dist = edit_distance(chunk, wl_ward)
                            if dist <= DIST_ALLOWED and dist < min_dist:
                                min_dist = dist
                                word = wl_ward
                                is_whitelist = True
                        if is_whitelist: ward = self.remove_special_characters(word)
                        else:
                          ward = self.remove_special_characters(chunk[-WINDOW_ALLOWED:])
                          for sw_ward in self.stopwords_dict["ward"]:
                            if should_elim_prefix_stopword(ward, sw_ward):
                                ward = ward[len(sw_ward):]
                    # Else, it is normal
                    else:
                        ward = self.remove_special_characters(chunk)
                    clear_chunks.append(ward)
                # If there are many commas, just append the rest of them
                if i > 2:
                    clear_chunks.append(chunk)
            return clear_chunks
        # B. COMMA = 1 (lack 1 comma)
        elif len(chunks) == 2:
          # B.1 Lack of comma to separate province and district
          ## Identify: If length of Province > WINDOW_ALLOWED and not in whitelist province
          if len(chunks[1]) > WINDOW_ALLOWED - 1:
            # print(1)
            # Whitelist Province flow
            is_whitelist_province = False
            # Remove stopwords from the Province
            chunk1 = self.remove_special_characters(chunks[1])
            for stopword in self.stopwords_dict["province"]:
                if should_elim_prefix_stopword(chunk1, stopword):
                    chunk1 = chunk1[len(stopword):]
                    break
            # Whitelist Check
            word, min_dist = "", float('inf')
            for wl_province in self.whitelisted_province:
                dist = edit_distance(chunk1, wl_province)
                if dist <= DIST_ALLOWED and dist < min_dist:
                    is_whitelist_province = True
                    word = wl_province
                    min_dist = dist
            if is_whitelist_province:
                province = self.remove_special_characters(word)
                clear_chunks.append(province)
                # If we can find a stopword, we can split from it
                is_separable_by_district_sw = False
                # Remove stopwords from the Ward & District
                chunk = self.remove_special_characters(chunks[0])
                for stopword in self.stopwords_dict["district"] + self.stopwords_dict["ward"]:
                    if should_elim_prefix_stopword(chunk, stopword):
                        chunk = chunk[len(stopword):]
                        break
                for stopword in self.stopwords_dict["district"]:
                    if stopword in chunk:
                        is_separable_by_district_sw = True
                        anchor = chunk.rfind(stopword)
                        if anchor == len(chunk) - len(stopword): # stopword is at the end -> not an actual stopword
                            if len(chunk) >= WINDOW_ALLOWED:
                                district = chunk[-len(chunk)//2 - OFFSET:]
                                ward = chunk[:-len(chunk)//2 + OFFSET]
                                clear_chunks.append(district)
                                clear_chunks.append(ward) if len(ward) <= WINDOW_ALLOWED else clear_chunks.append(ward[-WINDOW_ALLOWED:])
                                break
                            clear_chunks.append(chunk)
                            break
                        district = self.remove_special_characters(chunk[anchor+len(stopword):]) # district part
                        ward = self.remove_special_characters(chunk[:anchor]) # ward part
                        clear_chunks.append(district)
                        if len(ward) < 2: break
                        clear_chunks.append(ward) if len(ward) <= WINDOW_ALLOWED else clear_chunks.append(ward[-WINDOW_ALLOWED:])
                        break
                # Else, we can split from the last WINDOW_ALLOWED chars
                if not is_separable_by_district_sw:
                  if len(chunk) > WINDOW_ALLOWED:
                      district = chunk[-len(chunk)//2 - OFFSET:]
                      ward = chunk[:-len(chunk)//2 + OFFSET]
                      clear_chunks.append(district)
                      clear_chunks.append(ward) if len(ward) <= WINDOW_ALLOWED else clear_chunks.append(ward[-WINDOW_ALLOWED:])
                  else: clear_chunks.append(chunk)
                return clear_chunks
            # Non Whitelist Province flow
            # If we can find a stopword, we can split from it
            is_separable_by_province_sw = False
            for stopword in self.stopwords_dict["province"]:
                if stopword in chunks[1]:
                    is_separable_by_province_sw = True
                    anchor = chunks[1].rfind(stopword)
                    province = self.remove_special_characters(chunks[1][anchor+len(stopword):]) # province part
                    district = self.remove_special_characters(chunks[1][:anchor]) # district part
                    clear_chunks.append(province)
                    if len(district) > 2 and district not in self.whitelisted_district: clear_chunks.append(district)
            # Else, we can split from the last WINDOW_ALLOWED chars
            if not is_separable_by_province_sw:
              chunk = self.remove_special_characters(chunks[1])
              province = chunk[-len(chunk)//2 - OFFSET:]
              district = chunk[:-len(chunk)//2 + OFFSET]
              clear_chunks.append(province)
              if len(district) > 2 and district not in self.whitelisted_district: clear_chunks.append(district) # Filter trash words
            # We are in the Ward chunk, and it is longer than expected
            if len(chunks[0]) > WINDOW_ALLOWED:
              chunk = self.remove_special_characters(chunks[0])
              is_whitelist = False
              for wl_ward in self.whitelisted_ward:
                  if edit_distance(chunk, wl_ward) <= DIST_ALLOWED:
                      is_whitelist = True
                      next = self.remove_special_characters(wl_ward)
                      break
              if not is_whitelist: next = self.remove_special_characters(chunk[-WINDOW_ALLOWED:])
            else:
              next = self.remove_special_characters(chunks[0])
            clear_chunks.append(next)
            return clear_chunks

          # B.2 Lack of comma to separate ward and province
          ## Identify: If length of District > WINDOW_ALLOWED and not in whitelisted districts
          elif len(chunks[0]) > WINDOW_ALLOWED - 1:
            #print(2)
            # Append Province chunk first
            chunk = self.remove_special_characters(chunks[1])
            for stopword in self.stopwords_dict["province"]:
                if should_elim_prefix_stopword(chunk, stopword):
                    chunks[1] = chunks[1][len(stopword):]
                    break
            clear_chunks.append(self.remove_special_characters(chunks[1]))

            # Remove stopwords from the Ward & District
            chunk = self.remove_special_characters(chunks[0])
            for stopword in self.stopwords_dict["district"] + self.stopwords_dict["ward"]:
                if should_elim_prefix_stopword(chunk, stopword):
                    chunks[0] = chunks[0][len(stopword):]
                    break

            # If we can find a stopword, we can split from it
            is_separable_by_district_sw = False
            chunk = self.remove_special_characters(chunks[0])
            for stopword in self.stopwords_dict["district"]:
                if stopword in chunk:
                    is_separable_by_district_sw = True
                    anchor = chunk.rfind(stopword)
                    district = self.remove_special_characters(chunk[anchor+len(stopword):]) # district part
                    ward = self.remove_special_characters(chunk[:anchor]) # ward part
                    clear_chunks.append(district)
                    clear_chunks.append(ward) if len(ward) <= WINDOW_ALLOWED else clear_chunks.append(ward[-WINDOW_ALLOWED:])
            # Else, we can split from the last WINDOW_ALLOWED chars
            if not is_separable_by_district_sw:
              chunk = self.remove_special_characters(chunk)
              district = chunk[-len(chunk)//2 - OFFSET:]
              ward = chunk[:-len(chunk)//2 + OFFSET]
              clear_chunks.append(district)
              clear_chunks.append(ward) if len(ward) <= WINDOW_ALLOWED else clear_chunks.append(ward[-WINDOW_ALLOWED:])
            return clear_chunks

        # C. COMMA = 0
        elif len(chunks) == 1:
          # Flags
          is_separable_by_province_sw = False
          is_separable_by_district_sw = False
          chunks[0] = self.remove_special_characters(chunks[0])
          for stopword in self.stopwords_dict["province"]:
              if stopword in chunks[0]:
                  is_separable_by_province_sw = True
                  anchor = chunks[0].rfind(stopword)
                  province = self.remove_special_characters(chunks[0][anchor+len(stopword):]) # province part
                  ward_district = self.remove_special_characters(chunks[0][:anchor]) # district part
                  clear_chunks.append(province)

                  for s_stopword in self.stopwords_dict["district"]:
                      if s_stopword in ward_district:
                          is_separable_by_district_sw = True
                          anchor = ward_district.rfind(s_stopword)
                          district = self.remove_special_characters(ward_district[anchor+len(s_stopword):]) # district part
                          ward = self.remove_special_characters(ward_district[:anchor]) # ward part
                          if len(district) < 3: return clear_chunks
                          clear_chunks.append(district)
                          if len(ward) < 3: return clear_chunks
                          clear_chunks.append(ward) if len(ward) <= WINDOW_ALLOWED else clear_chunks.append(ward[-WINDOW_ALLOWED:])
                          return clear_chunks
                  if not is_separable_by_district_sw:
                      chunk = self.remove_special_characters(ward_district)
                      district = chunk[-WINDOW_ALLOWED:]
                      ward = chunk[:-WINDOW_ALLOWED]
                      if len(district) < 3: return clear_chunks
                      clear_chunks.append(district)
                      if len(ward) < 3: return clear_chunks
                      clear_chunks.append(ward) if len(ward) <= WINDOW_ALLOWED else clear_chunks.append(ward[-WINDOW_ALLOWED:])
                  return clear_chunks
          if not is_separable_by_province_sw:
              province = chunks[0][-WINDOW_ALLOWED:]
              ward_district = chunks[0][:-WINDOW_ALLOWED]
              clear_chunks.append(province)
              for s_stopword in self.stopwords_dict["district"]:
                  if s_stopword in ward_district:
                      is_separable_by_district_sw = True
                      anchor = ward_district.rfind(s_stopword)
                      district = self.remove_special_characters(ward_district[anchor+len(s_stopword):]) # district part
                      ward = self.remove_special_characters(ward_district[:anchor]) # ward part
                      if len(district) < 3: return clear_chunks
                      clear_chunks.append(district)
                      if len(ward) < 3: return clear_chunks
                      clear_chunks.append(ward) if len(ward) <= WINDOW_ALLOWED else clear_chunks.append(ward[-WINDOW_ALLOWED:])
                      return clear_chunks
              if not is_separable_by_district_sw:
                  chunk = self.remove_special_characters(ward_district)
                  district = chunk[-WINDOW_ALLOWED:]
                  ward = chunk[:-WINDOW_ALLOWED]
                  if len(district) < 3: return clear_chunks
                  clear_chunks.append(district)
                  if len(ward) < 3: return clear_chunks
                  clear_chunks.append(ward) if len(ward) <= WINDOW_ALLOWED else clear_chunks.append(ward[-WINDOW_ALLOWED:])
              return clear_chunks
        return ["", "", ""]

    def find_nearest_string(self, tree: BKTree, text: str, is_province=False):
        # Early exit if text is empty or too short and non-numeric.
        if not text or (len(text) < 3 and (not text.isdigit() or is_province)):
            return ""
        result = tree.search(text)
        result.sort(key=lambda x: edit_distance(x, text))
        ans = result[0] if len(result) > 0 else ""
        # Early exit if ans is empty or too short and non-numeric.
        if not ans or (len(text) < 3 and (not text.isdigit() or is_province)):
            return ""
        return ans

    def process(self, s: str):
        # write your process string here
        sanitized_address = self.remove_stopwords_from_address(s)
        # Final Normalization
        while len(sanitized_address) < 3: sanitized_address.append("")
        if len(sanitized_address) > 3: sanitized_address = sanitized_address[:3]
        # TODO: code to split string
        ward = self.find_nearest_string(self.root_ward, sanitized_address[2]) if len(sanitized_address) > 2 else ""
        district = self.find_nearest_string(self.root_district, sanitized_address[1]) if len(sanitized_address) > 1 else ""
        province = self.find_nearest_string(self.root_province, sanitized_address[0], True) if len(sanitized_address) > 0 else ""
        result = {
            "province": self.province_sanitized_map[province] if province != "" else "",
            "district": self.district_sanitized_map[district] if district != "" else "",
            "ward": self.ward_sanitized_map[ward] if ward != "" else ""
        }
        # TODO: only return result
        return result, sanitized_address

In [8]:
# NOTE: DO NOT change this cell
# This cell is for downloading private test
!rm -rf test.json
# this link is public test
!gdown --fuzzy https://drive.google.com/file/d/1PBt3U9I3EH885CDhcXspebyKI5Vw6uLB/view?usp=sharing -O test.json

Downloading...
From: https://drive.google.com/uc?id=1PBt3U9I3EH885CDhcXspebyKI5Vw6uLB
To: /content/test.json
100% 79.4k/79.4k [00:00<00:00, 3.36MB/s]


In [106]:
# NOTE: DO NOT change this cell
# This cell is for scoring

TEAM_NAME = 'DEFAULT_NAME'  # This should be your team name
EXCEL_FILE = f'{TEAM_NAME}.xlsx'

import json
import time
with open('test.json') as f:
    data = json.load(f)

# data = [
#     {
#         "text": "Khu phố 4 Thị trấn, Dương Minh Châu, Tây Ninh",
#         "result": {'province': 'Tây Ninh', 'district': 'Dương Minh Châu', 'ward': ''}
#     }
# ]

# Input:  Khu phố 4 Thị trấn, Dương Minh Châu, Tây Ninh
# Sanitized:  ['tayninh', 'duongminhchau', 'o4thitran']
# Result:  {'province': 'Tây Ninh', 'district': 'Dương Minh Châu', 'ward': 'Thới An'}
# Answer:  {'province': 'Tây Ninh', 'district': 'Dương Minh Châu', 'ward': ''}



summary_only = True
df = []
solution = Solution()
timer = []
correct = 0
ok_p, ok_d, ok_w = 0, 0, 0
for test_idx, data_point in enumerate(data):
    address = data_point["text"]
    ok = 0
    try:
        start = time.perf_counter_ns()
        result, sanitized = solution.process(address)
        answer = data_point["result"]
        finish = time.perf_counter_ns()
        timer.append(finish - start)
        ok += int(answer["province"] == result["province"])
        ok_p += int(answer["province"] == result["province"])
        ok += int(answer["district"] == result["district"])
        ok_d += int(answer["district"] == result["district"])
        ok_w += int(answer["ward"] == result["ward"])
        ok += int(answer["ward"] == result["ward"])
        df.append([
            test_idx,
            address,
            answer["province"],
            result["province"],
            int(answer["province"] == result["province"]),
            answer["district"],
            result["district"],
            int(answer["district"] == result["district"]),
            answer["ward"],
            result["ward"],
            int(answer["ward"] == result["ward"]),
            ok,
            timer[-1] / 1_000_000_000,
        ])
        # TODO: REMOVE
        if ok < 3:
          print("Input: ", address)
          print("Sanitized: ", sanitized)
          print("Result: ", result)
          print("Answer: ", {
              "province": answer["province"],
              "district": answer["district"],
              "ward": answer["ward"]
          })
          print("----")
    except Exception as e:
        print("*****Error*****", address, e, e.with_traceback)
        df.append([
            test_idx,
            address,
            answer["province"],
            "EXCEPTION",
            0,
            answer["district"],
            "EXCEPTION",
            0,
            answer["ward"],
            "EXCEPTION",
            0,
            0,
            0,
        ])
        # any failure count as a zero correct
        pass
    correct += ok


    if not summary_only:
        # responsive stuff
        print(f"Test {test_idx:5d}/{len(data):5d}")
        print(f"Correct: {ok}/3")
        print(f"Time Executed: {timer[-1] / 1_000_000_000:.4f}")

# print(province_correct)
print(f"-"*30)
total = len(data) * 3
score_scale_10 = round(correct / total * 10, 2)
if len(timer) == 0:
    timer = [0]
max_time_sec = round(max(timer) / 1_000_000_000, 4)
avg_time_sec = round((sum(timer) / len(timer)) / 1_000_000_000, 4)

import pandas as pd

df2 = pd.DataFrame(
    [[correct, total, score_scale_10, max_time_sec, avg_time_sec]],
    columns=['correct', 'total', 'score / 10', 'max_time_sec', 'avg_time_sec',],
)

columns = [
    'ID',
    'text',
    'province',
    'province_student',
    'province_correct',
    'district',
    'district_student',
    'district_correct',
    'ward',
    'ward_student',
    'ward_correct',
    'total_correct',
    'time_sec',
]

df = pd.DataFrame(df)
df.columns = columns
print('---- OVERTIME TESTS ----')
print(df[df['time_sec'] > 0.1][['text', 'time_sec']])
print('----')

print(f'{TEAM_NAME = }')
print(f'{EXCEL_FILE = }')
print(df2)
print("Province:", ok_p, "/", len(data))
print("District:", ok_d, "/", len(data))
print("Ward:", ok_w, "/", len(data))

!pip install xlsxwriter
writer = pd.ExcelWriter(EXCEL_FILE, engine='xlsxwriter')
df2.to_excel(writer, index=False, sheet_name='summary')
df.to_excel(writer, index=False, sheet_name='details')
writer.close()


Input:  284DBis Ng Văn Giáo, P3, Mỹ Tho, T.Giang.
Sanitized:  ['giang.', 'mytho', 'p3']
Result:  {'province': 'An Giang', 'district': 'Mỹ Tho', 'ward': ''}
Answer:  {'province': 'Tiền Giang', 'district': 'Mỹ Tho', 'ward': '3'}
----
Input:  Khu phố 4 Thị trấn, Dương Minh Châu, Tây Ninh
Sanitized:  ['tayninh', 'duongminhchau', 'tran']
Result:  {'province': 'Tây Ninh', 'district': 'Dương Minh Châu', 'ward': 'Tự An'}
Answer:  {'province': 'Tây Ninh', 'district': 'Dương Minh Châu', 'ward': ''}
----
Input:  Tiểu khu 3, thị trấn Ba Hàng, huyện Phổ Yên, tỉnh Thái Nguyên.
Sanitized:  ['thainguyen.', 'phoyen', 'ranbahang']
Result:  {'province': 'Thái Nguyên', 'district': 'Phổ Yên', 'ward': ''}
Answer:  {'province': '', 'district': '', 'ward': ''}
----
Input:  Xóm 2, Hưng Mỹ, Hưng Nguyên, Nghệ An
Sanitized:  ['nghean', 'gnguyen', 'hungngu']
Result:  {'province': 'Nghệ An', 'district': 'Hưng Nguyên', 'ward': 'Hưng Hà'}
Answer:  {'province': 'Nghệ An', 'district': 'Hưng Nguyên', 'ward': 'Hưng Mỹ'}
